# Tides Module

**This is a literate notebook.**

The goal of this module is to provide a fast and simple interface to find tide predictions.

For folks who would like to better understand the physics of tides here is an [Awesome Accessible Description of Tides](http://faculty.washington.edu/pmacc/LO/tides_background.html)


## NOAA tides data

Ultimately NOAA is the source of all information about tides here in the US. They maintain a network for 3000+ tide stations for which tide predictions are available.  

Super helpful introductory page: https://tidesandcurrents.noaa.gov/PageHelp.html, and a page containing dozens of references about tides and how they work https://tidesandcurrents.noaa.gov/education.html.

NOAA has a ton of tides data.  And NOAA has been around for a long time.  As a result the interfaces are a bit duplicative.  The best, most modern, interface is a web service which allows you to pull down a CSV of tide predictions (and a ton of other info).

A URL generator for this web service is here:  https://tidesandcurrents.noaa.gov/api-helper/url-generator.html  Using these pages, you can fill in a form and it hands you back a URL which can be used to pull the requested data.

For example this URL:

`https://tidesandcurrents.noaa.gov/api/datagetter?date=today&station=9447130&product=water_level&datum=MLLW&time_zone=gmt&units=english&format=csv`

Returns a CSV showing the tides in Seattle (station 9447130) for the current day (`today`) using GMT times and relative to the MLLW datum (mean low low water).  MLLW is the standard used by NOAA printed charts (this [page](https://noaanhc.wordpress.com/tag/mllw/#:~:text=Mean%20Lower%20Low%20Water%20(MLLW)%20is%20simply%20the%20lowest%20of,runs%20from%201983%20through%202001.) has a nice discussion of MLLW and related datums).

The result is here (just the first lines):

```
Date Time, Water Level, Sigma, O or I (for verified), F, R, L, Quality 
2020-06-09 00:00,5.172,0.098,0,0,0,0,p
2020-06-09 00:06,5.494,0.102,0,0,0,0,p
2020-06-09 00:12,5.805,0.085,0,0,0,0,p
2020-06-09 00:18,6.104,0.105,0,0,0,0,p
2020-06-09 00:24,6.412,0.098,0,0,0,0,p
2020-06-09 00:30,6.714,0.105,0,0,0,0,p
2020-06-09 00:36,7.013,0.095,0,0,0,0,p
```

Note, you can only request these predictions for harmonic stations (which are first class), and not subordinate stations.  At harmonic stations NOAA has built a model for predicted tide height as a sum of many sine waves, each based on some physical phenomena (like the position of the moon).  Subordinate stations (which are much more numerous) are each subordinate to some harmonic station (their reference).  Subordinate stations only have time offsets for high/low (in minutes) and multipliers for high/low tides.

- A list of all stations
  - https://tidesandcurrents.noaa.gov/stations.html
- Additional info on Harmonic Constituents
  - https://tidesandcurrents.noaa.gov/about_harmonic_constituents.html
  - https://beta.tidesandcurrents.noaa.gov/glossary.html
- Harmonic constituents for Seattle:
  - https://tidesandcurrents.noaa.gov/harcon.html?unit=1&timezone=0&id=9447130&name=Seattle&state=WA
  - https://tidesandcurrents.noaa.gov/mdapi/latest/webapi/stations/9447130.json?expand=harcon&units=english


After some digging around I found a way to fetch the offsets for a subordinate station, in this case Gig Harbor, station 9447130.  First you need I needed to fetch this URL from the Metadata API.  

https://tidesandcurrents.noaa.gov/mdapi/latest/webapi/stations/9446369.json?units=english

And then buried in the results was a URL that showed the actual offsets:

https://tidesandcurrents.noaa.gov/mdapi/v1.0/webapi/stations/9446369/tidepredoffsets.json

```
{
  "refStationId": "9447130",
  "type": "S",
  "heightOffsetHighTide": 1.05,
  "heightOffsetLowTide": 0.99,
  "timeOffsetHighTide": 18,
  "timeOffsetLowTide": 22,
  "heightAdjustedType": "R",
  "self": null
}
```

## Currents

NOAA puts similar efforts into Currents.  A map of NOAA current prediction stations is available here:

https://tidesandcurrents.noaa.gov/map/index.html?type=CurrentPredictions

The complete list of Washington Current stations is here:

https://tidesandcurrents.noaa.gov/noaacurrents/Stations?g=698

Note, much like tides, currents are estimated using harmonic methods (though I am not sure they are identical).  Historic data is collected for weeks (e.g. at West Point from 2015-07-27 20:02:00 to 2015-09-10 20:46:00).  Data is fit to harmonic models and then predictions are available (much as tide predictions are available).  Unfortunately the prediction models appear to estimate current speed, but only two directions are given, one for ebb and another for flood.  

There are currently no active stations in Puget Sound (an active station has data collection ongoing in the field).  There have been many historical current surveys, and these include locations in Puget Sound.  The list of these surveys is here.

https://tidesandcurrents.noaa.gov/cdata/StationList?type=Current+Data&filter=historic&pid=36

A bit of historic data for West Point is here:

https://tidesandcurrents.noaa.gov/cdata/DataPlot?id=PUG1515&bin=15&bdate=20150806&edate=20150807&unit=1&timeZone=UTC

And this is a direct download of 31 days of data as a CSV.  

https://tidesandcurrents.noaa.gov/cdata/DataPlot?id=PUG1515&bin=15&bdate=20150801&edate=20150905&unit=1&timeZone=UTC&view=csv

It would be great to create a simple program that could enumerate all the current stations in the area and then download all the raw data.  From that we could build models both for current speed and direction.

Note the there is an annoying problem in that you can only request up to 31 days of data(why?) and you can only request dates for which data is available.  The problem is that I cannot find a programmatic way to determine the valid dates.  The only way is by reading a web page, and that web page is only available from a Javascript.




# Tides

This notebook is closely related to the Current_Module.  Please take a look there as well.




## Approach

Loads a cached set of NOAA tide prediction data and then looks up the closest tide for
any time.


## TODO

- Figure out the harmonics...  much more compact.
  - This will help with analysis of currents as well.

In [2]:
import pandas as pd
import numpy as np
import os
import os.path
import urllib

In [3]:
# These are libraries written for RegattaAnalysis
from global_variables import G  # global variables

G.init_seattle()

In [4]:
TIDES_PATH = os.path.join(G.DATA_DIRECTORY, 'Tides/seattle_tides2.pd')
TIDES_DF = pd.read_pickle(G.TIDES_PATH)

In [5]:
TIDES_DF

,prediction,date_time
0,6.309,2018-12-31 16:00:00-08:00
1,6.055,2018-12-31 16:06:00-08:00
2,5.803,2018-12-31 16:12:00-08:00
3,5.551,2018-12-31 16:18:00-08:00
4,5.301,2018-12-31 16:24:00-08:00
...,...,...
263035,10.402,2021-12-31 15:30:00-08:00
263036,10.199,2021-12-31 15:36:00-08:00
263037,9.988,2021-12-31 15:42:00-08:00
263038,9.768,2021-12-31 15:48:00-08:00


In [ ]:
def tides_at_closest(times):
    base_time = TIDES_DF.date_time.iloc[0]
    last_time = TIDES_DF.date_time.iloc[-1]    
    if (times < base_time).any():
        raise Exception("Tide time too early {0}".format(times.min()))
    if (times > last_time).any():
        raise Exception("Time time too late {0}".format(times.max()))
    # Compute the nearest ti
    tide_index = ((times - base_time).dt.total_seconds() / 360).round().astype(np.int)
    return TIDES_DF.iloc[tide_index].prediction.values